<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/Chapter-2/Chapter2_descriptive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# setup and installation
!pip install vaex
!pip install stargazer
!pip install missingno

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')

import os
path = '/content/drive/My Drive/2 Essay Bayesian dinamic model for vaccine hesitancy/Data'
os.chdir(path)

project_id = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project_id, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

import pandas as pd
import vaex
import numpy as np
import matplotlib.pyplot as plt

import missingno as ms
import seaborn as sns

%matplotlib inline

from stargazer.stargazer import Stargazer

SEED = np.random.seed(42)

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('df_descriptive.csv')
df.head(5)

,diff_dose1_dose2,diff_dose2_dose3,n_doses,dose1_ASTRAZENECA,dose1_JANSSEN,dose1_PFIZER,dose1_SINOVAC,dose2_ASTRAZENECA,dose2_JANSSEN,dose2_PFIZER,...,scaled_mean_literacy,scaled_mean_families,scaled_votos_PSL-PT_2018,id_municipio_endereco_paciente,bairros,idade_paciente,mean_income_avg,mean_literacy,mean_families,votos_PSL-PT_2018
0,157,0,3,0,1,0,0,0,1,0,...,-0.169016,0.735782,-1.682533,4314902,['RESTINGA'],37,1573.440,96.000000,3.440000,-0.096122
1,189,0,3,0,1,0,0,0,1,0,...,0.567233,0.031636,1.454153,3304557,"['MARECHAL HERMES', 'DEODORO', 'VILA MILITAR']",41,2222.860,97.866667,3.153333,0.246763
2,168,0,3,0,1,0,0,0,1,0,...,0.895916,-1.118742,-0.909994,4314902,"['VILA IPIRANGA', 'JARDIM EUROPA', 'VILA JARDIM']",47,3546.250,98.700000,2.685000,-0.011672
3,162,0,3,0,1,0,0,0,1,0,...,0.817032,0.170828,-0.166758,4106902,"['UBERABA', 'JARDIM DAS AMÉRICAS']",41,4628.515,98.500000,3.210000,0.069574
4,172,0,3,0,1,0,0,0,1,0,...,0.619822,0.023449,-0.216864,4106902,['PILARZINHO'],48,3506.250,98.000000,3.150000,0.064097


In [ ]:
# Create the 'dose1' column based on the dose1_* columns
df['dose1'] = ''
df.loc[df['dose1_ASTRAZENECA'] == 1, 'dose1'] = 'ASTRAZENECA'
df.loc[df['dose1_JANSSEN'] == 1, 'dose1'] = 'JANSSEN'
df.loc[df['dose1_PFIZER'] == 1, 'dose1'] = 'PFIZER'
df.loc[df['dose1_SINOVAC'] == 1, 'dose1'] = 'SINOVAC'

# Create the 'dose3' column based on the dose3_* columns
df['dose3'] = ''
df.loc[df['dose3_ASTRAZENECA'] == 1, 'dose3'] = 'ASTRAZENECA'
df.loc[df['dose3_JANSSEN'] == 1, 'dose3'] = 'JANSSEN'
df.loc[df['dose3_PFIZER'] == 1, 'dose3'] = 'PFIZER'
df.loc[df['dose3_SINOVAC'] == 1, 'dose3'] = 'SINOVAC'

In [ ]:
# Set up the BigQuery client
client = bigquery.Client(project=project_id)

# Define the query
query = """
SELECT 
  m.id_municipio,
  m.id_municipio_tse,
  m.nome, 
  m.sigla_uf, 
  m.nome_regiao, 
  p.populacao
FROM 
  `basedosdados.br_bd_diretorios_brasil.municipio` AS m
JOIN 
  `basedosdados.br_ibge_populacao.municipio` AS p
ON 
  m.id_municipio = p.id_municipio
JOIN (
  SELECT 
    id_municipio, 
    MAX(ano) AS max_ano 
  FROM 
    `basedosdados.br_ibge_populacao.municipio` 
  GROUP BY 
    id_municipio
) AS max_ano
ON 
  p.id_municipio = max_ano.id_municipio 
  AND p.ano = max_ano.max_ano
"""

# Execute the query and store the results in a dataframe
df2 = client.query(query).to_dataframe()
df2.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516
1,1100023,78,Ariquemes,RO,Norte,111148
2,1100031,450,Cabixi,RO,Norte,5067
3,1100049,94,Cacoal,RO,Norte,86416
4,1100056,272,Cerejeiras,RO,Norte,16088


In [ ]:
df2['id_municipio'] = df2['id_municipio'].astype('int64')
df = pd.merge(df, df2, left_on='id_municipio_endereco_paciente', right_on='id_municipio')

In [ ]:
# Replace 0 values with NaN in the 'diff_dose1_dose2' column
df['diff_dose1_dose2'] = df['diff_dose1_dose2'].replace(0, np.nan)

# Replace 0 values with NaN in the 'diff_dose2_dose3' column
df['diff_dose2_dose3'] = df['diff_dose2_dose3'].replace(0, np.nan)



In [ ]:
df = vaex.from_pandas(df)
df.head(5)

#,diff_dose1_dose2,diff_dose2_dose3,n_doses,dose1_ASTRAZENECA,dose1_JANSSEN,dose1_PFIZER,dose1_SINOVAC,dose2_ASTRAZENECA,dose2_JANSSEN,dose2_PFIZER,dose2_SINOVAC,dose3_ASTRAZENECA,dose3_JANSSEN,dose3_PFIZER,dose3_SINOVAC,age_65_80,age_80_plus,comorbidities,security_forces,health_professionals,sexo_paciente_F,raca_cor_white,raca_cor_black,raca_cor_brown,scaled_idade_paciente,scaled_mean_income_avg,scaled_mean_literacy,scaled_mean_families,scaled_votos_PSL-PT_2018,id_municipio_endereco_paciente,bairros,idade_paciente,mean_income_avg,mean_literacy,mean_families,votos_PSL-PT_2018,dose1,dose3,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao
0,157,nan,3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.290064,-0.737613,-0.169016,0.735782,-1.68253,4314902,['RESTINGA'],37,1573.44,96,3.44,-0.0961222,JANSSEN,,4314902,88013,Porto Alegre,RS,Sul,1492530
1,168,nan,3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.246818,0.242897,0.895916,-1.11874,-0.909994,4314902,"['VILA IPIRANGA', 'JARDIM EUROPA', 'VILA JARDIM']",47,3546.25,98.7,2.685,-0.0116725,JANSSEN,,4314902,88013,Porto Alegre,RS,Sul,1492530
2,168,nan,3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.407883,0.107136,0.343729,-0.750293,-2.09768,4314902,"""['JARDIM DO SALSO', 'JARDIM CARVALHO', 'BOM JES...",50,3273.1,97.3,2.835,-0.141504,JANSSEN,,4314902,88013,Porto Alegre,RS,Sul,1492530
3,179,nan,3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-0.182687,2.21717,1.10101,-1.14085,-0.87775,4314902,"""['BOA VISTA', 'VILA IPIRANGA', 'JARDIM EUROPA',...",39,7518.53,99.22,2.676,-0.00814779,JANSSEN,,4314902,88013,Porto Alegre,RS,Sul,1492530
4,2,nan,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.139442,0.682544,0.77759,-2.56797,-1.43265,4314902,"['CENTRO HISTÓRICO', 'PRAIA DE BELAS']",45,4430.83,98.4,2.095,-0.0688061,ASTRAZENECA,,4314902,88013,Porto Alegre,RS,Sul,1492530


In [ ]:
# df = df.join(df2, how='inner', left_on ='id_municipio_endereco_paciente', right_on='id_municipio')

--------------------------------------------------------
Descriptive statistics

In [ ]:
df.info()

column,type,unit,description,expression
diff_dose1_dose2,float64,,,
diff_dose2_dose3,float64,,,
n_doses,int64,,,
dose1_ASTRAZENECA,int64,,,
dose1_JANSSEN,int64,,,
dose1_PFIZER,int64,,,
dose1_SINOVAC,int64,,,
dose2_ASTRAZENECA,int64,,,
dose2_JANSSEN,int64,,,
dose2_PFIZER,int64,,,


In [ ]:
df.column_names

['diff_dose1_dose2',
 'diff_dose2_dose3',
 'n_doses',
 'dose1_ASTRAZENECA',
 'dose1_JANSSEN',
 'dose1_PFIZER',
 'dose1_SINOVAC',
 'dose2_ASTRAZENECA',
 'dose2_JANSSEN',
 'dose2_PFIZER',
 'dose2_SINOVAC',
 'dose3_ASTRAZENECA',
 'dose3_JANSSEN',
 'dose3_PFIZER',
 'dose3_SINOVAC',
 'age_65_80',
 'age_80_plus',
 'comorbidities',
 'security_forces',
 'health_professionals',
 'sexo_paciente_F',
 'raca_cor_white',
 'raca_cor_black',
 'raca_cor_brown',
 'scaled_idade_paciente',
 'scaled_mean_income_avg',
 'scaled_mean_literacy',
 'scaled_mean_families',
 'scaled_votos_PSL-PT_2018',
 'id_municipio_endereco_paciente',
 'bairros',
 'idade_paciente',
 'mean_income_avg',
 'mean_literacy',
 'mean_families',
 'votos_PSL-PT_2018',
 'dose1',
 'dose3',
 'id_municipio',
 'id_municipio_tse',
 'nome',
 'sigla_uf',
 'nome_regiao',
 'populacao']

In [ ]:
df['nome_regiao'].unique()

['Sul', 'Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste']

In [ ]:
df[['n_doses','diff_dose1_dose2', 'diff_dose2_dose3','idade_paciente',
    'mean_income_avg', 'mean_literacy', 'mean_families', 'votos_PSL-PT_2018']
   ].describe()

,n_doses,diff_dose1_dose2,diff_dose2_dose3,idade_paciente,mean_income_avg,mean_literacy,mean_families,votos_PSL-PT_2018
data_type,int64,float64,float64,int64,float64,float64,float64,float64
count,17926966,14494789,3672308,17926966,15625791,15625791,15625791,15046357
NA,0,3432177,14254658,0,2301175,2301175,2301175,2880609
mean,2.0729894283282513,70.89746984243786,165.55798914470137,42.402747402990556,3057.5352567569576,96.42851768552846,3.140453757974791,0.08780302193811203
std,0.637835,42.574024,41.504625,18.626061,2012.023031,2.535375,0.407113,0.109314
min,1,1.0,1.0,-16,567.34,75.6,1.73,-0.354076
max,4,30445.0,354.0,221,18553.01,99.8,4.72,0.55971


In [ ]:
# Select the desired columns and calculate summary statistics
df_selected = df[['n_doses', 'diff_dose1_dose2', 'diff_dose2_dose3', 'idade_paciente',
                  'mean_income_avg', 'mean_literacy', 'mean_families', 'votos_PSL-PT_2018']].describe()

# Transpose the DataFrame
df_transposed = df_selected.T

# Convert the DataFrame to a LaTeX table
latex_table = df_transposed.to_latex()

# Print the LaTeX table
print(latex_table)

\begin{tabular}{llllllll}
\toprule
{} & data\_type &     count &        NA &                 mean &          std &       min &       max \\
\midrule
n\_doses           &     int64 &  17926966 &         0 &   2.0729894283282513 &     0.637835 &         1 &         4 \\
diff\_dose1\_dose2  &   float64 &  14494789 &   3432177 &    70.89746984243786 &    42.574024 &       1.0 &   30445.0 \\
diff\_dose2\_dose3  &   float64 &   3672308 &  14254658 &   165.55798914470137 &    41.504625 &       1.0 &     354.0 \\
idade\_paciente    &     int64 &  17926966 &         0 &   42.402747402990556 &    18.626061 &       -16 &       221 \\
mean\_income\_avg   &   float64 &  15625791 &   2301175 &    3057.535256756094 &  2012.023031 &    567.34 &  18553.01 \\
mean\_literacy     &   float64 &  15625791 &   2301175 &     96.4285176855247 &     2.535375 &      75.6 &      99.8 \\
mean\_families     &   float64 &  15625791 &   2301175 &   3.1404537579747798 &     0.407113 &      1.73 &      4.72 \\
votos\_P

<ipython-input-15-cb50a4fa6cfc>:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df_transposed.to_latex()


In [ ]:
# # Get a list of the unique regions
# regions = df['nome_regiao'].unique()

# # Loop over the regions and get the described DataFrame for each one
# described_dfs = []
# for region in regions:
#     # Filter the DataFrame to only include the current region
#     region_df = df[df['nome_regiao'] == region]
    
#     # Get the described DataFrame for the current region
#     described_df = region_df[['n_doses','diff_dose1_dose2', 'diff_dose2_dose3','idade_paciente',
#                               'mean_income_avg', 'mean_literacy', 'mean_families', 'votos_PSL-PT_2018']
#                              ].describe()
    
#     # Add the described DataFrame to the list of results
#     described_dfs.append(described_df)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Create separate dataframes for each region
dfs = []
for region in df['nome_regiao'].unique():
    region_df = df[df['nome_regiao'] == region]
    dfs.append(region_df)

# Compute summary statistics for each dataframe
summary_dfs = []
for region_df in dfs:
    summary_df = region_df[['n_doses','diff_dose1_dose2', 'diff_dose2_dose3','idade_paciente',
                            'mean_income_avg', 'votos_PSL-PT_2018']].describe().round(2)
    summary_df = summary_df.transpose()
    summary_df['region'] = region_df['nome_regiao'].unique()[0]
    summary_dfs.append(summary_df)

# Concatenate all summary dataframes into a single table
summary_table = pd.concat(summary_dfs)

# Reorder columns and reset index
summary_table = summary_table[['region', 'count', 'NA', 'mean', 'std', 'min', 'max']].reset_index()
summary_table = summary_table.round({'mean': 2, 'std': 2, 'min': 2, 'max': 2})

# Rename index column to variable name
summary_table = summary_table.rename(columns={'index': 'variable'})
summary_table['mean'] = summary_table['mean'].apply('{:,.2f}'.format)

# Print summary table
print(summary_table.to_latex())

\begin{tabular}{lllllllll}
\toprule
{} &           variable &        region &    count &       NA &      mean &     std &     min &      max \\
\midrule
0  &            n\_doses &           Sul &  3674829 &        0 &      2.17 &    0.62 &       1 &        4 \\
1  &   diff\_dose1\_dose2 &           Sul &  3170890 &   503939 &     68.70 &   32.37 &    1.00 &   477.00 \\
2  &   diff\_dose2\_dose3 &           Sul &   940181 &  2734648 &    173.28 &   38.53 &    1.00 &   337.00 \\
3  &     idade\_paciente &           Sul &  3674829 &        0 &     43.46 &   18.71 &       0 &      129 \\
4  &    mean\_income\_avg &           Sul &  3218122 &   456707 &  3,660.20 & 1832.25 & 1277.48 & 13229.38 \\
5  &  votos\_PSL-PT\_2018 &           Sul &  3053702 &   621127 &      0.02 &    0.08 &   -0.20 &     0.25 \\
6  &            n\_doses &  Centro-Oeste &  1617560 &        0 &      2.09 &    0.65 &       1 &        4 \\
7  &   diff\_dose1\_dose2 &  Centro-Oeste &  1285143 &   332417 &     67.83 &   

<ipython-input-17-ca90229d49d5>:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summary_table.to_latex())


In [ ]:
# Create separate dataframes for each region
dfs = []
for city in df['nome'].unique():
    city_df = df[df['nome'] == city]
    dfs.append(city_df)

# Compute summary statistics for each dataframe
summary_dfs2 = []
for city_df in dfs:
    summary_df2 = city_df[['n_doses','diff_dose1_dose2', 'diff_dose2_dose3','idade_paciente',
                            'mean_income_avg', 'votos_PSL-PT_2018']].describe().round(2)
    summary_df2['city'] = city_df['nome'].unique()[0]
    summary_dfs2.append(summary_df2)

# Concatenate all summary dataframes into a single table
summary_table2 = pd.concat(summary_dfs2)

# Reorder columns and reset index
# summary_table2 = summary_table2[['count', 'mean']].reset_index()

# Rename index column to variable name
# summary_table2 = summary_table2.rename(columns={'index': 'variable'})
# summary_table2['mean'] = summary_table2['mean'].apply('{:,.2f}'.format)

In [ ]:
# select rows that have 'mean' in the index
mean_rows = summary_table2[summary_table2.index.str.contains('mean')]

# select rows that have 'std' in the index
std_rows = summary_table2[summary_table2.index.str.contains('std')]

# combine the two selections
selected_rows = pd.concat([mean_rows, std_rows])

# Create a new column indicating whether each row represents the mean or standard deviation
selected_rows['mean_or_std'] = np.where(selected_rows.index.str.contains('mean'), 'mean', 'std')

# Sort the dataframe by the city column and then by the mean_or_std column
selected_rows = selected_rows.sort_values(by=['city', 'mean_or_std'])

# Drop the mean_or_std column if it's not needed anymore
selected_rows = selected_rows.drop(columns=['mean_or_std'])

# Convert Numpy arrays to Pandas dataframes
selected_rows = pd.DataFrame(selected_rows)

# # Round every value in the dataframe to 2 decimal places
# selected_rows[['n_doses',	'diff_dose1_dose2',	'diff_dose2_dose3',	'idade_paciente', 'mean_income_avg',
#                'votos_PSL-PT_2018']] = selected_rows[['n_doses',	'diff_dose1_dose2',	'diff_dose2_dose3',
#                                                       'idade_paciente', 'mean_income_avg','votos_PSL-PT_2018']].apply('{:,.2f}'.format)

selected_rows[['n_doses',	'diff_dose1_dose2',	'diff_dose2_dose3',	'idade_paciente', 'mean_income_avg',
               'votos_PSL-PT_2018']] = selected_rows[['n_doses',	'diff_dose1_dose2',	'diff_dose2_dose3',
                                                      'idade_paciente', 'mean_income_avg','votos_PSL-PT_2018']].astype('float64')

selected_rows = selected_rows.round(2)

In [ ]:
# Define the query
query = """
SELECT 
  m.id_municipio,
  m.id_municipio_tse, 
  m.nome, 
  m.sigla_uf, 
  m.nome_regiao, 
  p.populacao
FROM 
  `basedosdados.br_bd_diretorios_brasil.municipio` AS m
JOIN 
  `basedosdados.br_ibge_populacao.municipio` AS p
ON 
  m.id_municipio = p.id_municipio
JOIN (
  SELECT 
    id_municipio, 
    MAX(ano) AS max_ano 
  FROM 
    `basedosdados.br_ibge_populacao.municipio` 
  GROUP BY 
    id_municipio
) AS max_ano
ON 
  p.id_municipio = max_ano.id_municipio 
  AND p.ano = max_ano.max_ano
"""

# Execute the query and store the results in a dataframe
df3 = client.query(query).to_dataframe()
df3.head(5)


,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516
1,1100023,78,Ariquemes,RO,Norte,111148
2,1100031,450,Cabixi,RO,Norte,5067
3,1100049,94,Cacoal,RO,Norte,86416
4,1100056,272,Cerejeiras,RO,Norte,16088


In [ ]:
# Merge the two dataframes on the 'city' column
selected_rows = selected_rows.merge(df3[['nome', 'sigla_uf', 'nome_regiao', 'populacao']], left_on='city', right_on='nome')

# Drop the 'nome' column if it is not needed anymore
selected_rows = selected_rows.drop(columns='nome')

selected_rows

,n_doses,diff_dose1_dose2,diff_dose2_dose3,idade_paciente,mean_income_avg,votos_PSL-PT_2018,city,sigla_uf,nome_regiao,populacao
0,1.84,76.29,178.40,36.71,2250.80,-0.09,Abaetetuba,PA,Norte,160439
1,0.53,27.54,46.99,17.22,1.57,0.00,Abaetetuba,PA,Norte,160439
2,2.25,62.86,151.40,41.35,3307.81,0.11,Campo Grande,RN,Nordeste,9686
3,2.25,62.86,151.40,41.35,3307.81,0.11,Campo Grande,AL,Nordeste,9576
4,2.25,62.86,151.40,41.35,3307.81,0.11,Campo Grande,MS,Centro-Oeste,916001
5,0.67,33.59,34.41,18.18,2181.17,0.05,Campo Grande,RN,Nordeste,9686
6,0.67,33.59,34.41,18.18,2181.17,0.05,Campo Grande,AL,Nordeste,9576
7,0.67,33.59,34.41,18.18,2181.17,0.05,Campo Grande,MS,Centro-Oeste,916001
8,2.12,73.68,174.24,42.72,3125.33,0.02,Caxias do Sul,RS,Sul,523716
9,0.61,29.75,35.61,18.28,764.63,0.01,Caxias do Sul,RS,Sul,523716


In [ ]:
# Print summary table
print(selected_rows.to_latex())

\begin{tabular}{lrrrrrrlllr}
\toprule
{} &  n\_doses &  diff\_dose1\_dose2 &  diff\_dose2\_dose3 &  idade\_paciente &  mean\_income\_avg &  votos\_PSL-PT\_2018 &              city & sigla\_uf &   nome\_regiao &  populacao \\
\midrule
0  &     1.84 &             76.29 &            178.40 &           36.71 &          2250.80 &              -0.09 &        Abaetetuba &       PA &         Norte &     160439 \\
1  &     0.53 &             27.54 &             46.99 &           17.22 &             1.57 &               0.00 &        Abaetetuba &       PA &         Norte &     160439 \\
2  &     2.25 &             62.86 &            151.40 &           41.35 &          3307.81 &               0.11 &      Campo Grande &       RN &      Nordeste &       9686 \\
3  &     2.25 &             62.86 &            151.40 &           41.35 &          3307.81 &               0.11 &      Campo Grande &       AL &      Nordeste &       9576 \\
4  &     2.25 &             62.86 &            151.40 &           

<ipython-input-22-1c9bf59663b5>:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(selected_rows.to_latex())


In [ ]:
selected_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   n_doses            50 non-null     float64
 1   diff_dose1_dose2   50 non-null     float64
 2   diff_dose2_dose3   50 non-null     float64
 3   idade_paciente     50 non-null     float64
 4   mean_income_avg    50 non-null     float64
 5   votos_PSL-PT_2018  50 non-null     float64
 6   city               50 non-null     object 
 7   sigla_uf           50 non-null     object 
 8   nome_regiao        50 non-null     object 
 9   populacao          50 non-null     Int64  
dtypes: Int64(1), float64(6), object(3)
memory usage: 4.3+ KB


In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Create separate dataframes for each vaccine
dfs = []
for vaccine in df['dose1'].unique():
    vaccine_df = df[df['dose1'] == vaccine]
    dfs.append(vaccine_df)

# Compute summary statistics for each dataframe
summary_dfs = []
for vaccine_df in dfs:
    summary_df = vaccine_df[['n_doses','diff_dose1_dose2', 'diff_dose2_dose3','idade_paciente',
                            'mean_income_avg', 'votos_PSL-PT_2018']].describe().round(2)
    summary_df = summary_df.transpose()
    summary_df['Dose 1'] = vaccine_df['dose1'].unique()[0]
    summary_dfs.append(summary_df)

# Concatenate all summary dataframes into a single table
summary_table = pd.concat(summary_dfs)

# Reorder columns and reset index
summary_table = summary_table[['Dose 1', 'count', 'NA', 'mean', 'std', 'min', 'max']].reset_index()
summary_table = summary_table.round({'mean': 2, 'std': 2, 'min': 2, 'max': 2})

# Rename index column to variable name
summary_table = summary_table.rename(columns={'index': 'variable'})
summary_table['mean'] = summary_table['mean'].apply('{:,.2f}'.format)



In [ ]:
# Print summary table
print(summary_table.to_latex())

\begin{tabular}{lllllllll}
\toprule
{} &           variable &       Dose 1 &    count &       NA &      mean &     std &    min &      max \\
\midrule
0  &            n\_doses &  ASTRAZENECA &  6618744 &        0 &      2.13 &    0.57 &      1 &        4 \\
1  &   diff\_dose1\_dose2 &  ASTRAZENECA &  5910150 &   708594 &     87.31 &   37.72 &   1.00 & 30445.00 \\
2  &   diff\_dose2\_dose3 &  ASTRAZENECA &  1475029 &  5143715 &    142.16 &   30.48 &   1.00 &   345.00 \\
3  &     idade\_paciente &  ASTRAZENECA &  6618744 &        0 &     48.12 &   14.73 &    -16 &      170 \\
4  &    mean\_income\_avg &  ASTRAZENECA &  5799578 &   819166 &  3,099.15 & 2036.72 & 567.34 & 18553.01 \\
5  &  votos\_PSL-PT\_2018 &  ASTRAZENECA &  5572129 &  1046615 &      0.09 &    0.11 &  -0.35 &     0.56 \\
6  &            n\_doses &      JANSSEN &   499404 &        0 &      3.00 &    0.02 &      3 &        4 \\
7  &   diff\_dose1\_dose2 &      JANSSEN &   109826 &   389578 &    170.42 &   17.81 &   1.00 & 

<ipython-input-30-83d34eb178f5>:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summary_table.to_latex())


In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Create separate dataframes for each vaccine
dfs = []
for vaccine in df['dose3'].unique():
    vaccine_df = df[df['dose3'] == vaccine]
    dfs.append(vaccine_df)

# Compute summary statistics for each dataframe
summary_dfs = []
for vaccine_df in dfs:
    summary_df = vaccine_df[['n_doses','diff_dose1_dose2', 'diff_dose2_dose3','idade_paciente',
                            'mean_income_avg', 'votos_PSL-PT_2018']].describe().round(2)
    summary_df = summary_df.transpose()
    summary_df['Dose 3'] = vaccine_df['dose3'].unique()[0]
    summary_dfs.append(summary_df)

# Concatenate all summary dataframes into a single table
summary_table = pd.concat(summary_dfs)

# Reorder columns and reset index
summary_table = summary_table[['Dose 3', 'count', 'NA', 'mean', 'std', 'min', 'max']].reset_index()
summary_table = summary_table.round({'mean': 2, 'std': 2, 'min': 2, 'max': 2})

# Rename index column to variable name
summary_table = summary_table.rename(columns={'index': 'variable'})
summary_table['mean'] = summary_table['mean'].apply('{:,.2f}'.format)

# Print summary table
print(summary_table.to_latex())

\begin{tabular}{lllllllll}
\toprule
{} &           variable &       Dose 3 &     count &        NA &      mean &     std &    min &      max \\
\midrule
0  &            n\_doses &  ASTRAZENECA &    214070 &         0 &      3.08 &    0.28 &      3 &        4 \\
1  &   diff\_dose1\_dose2 &  ASTRAZENECA &    199525 &     14545 &     70.34 &   35.25 &   1.00 & 11443.00 \\
2  &   diff\_dose2\_dose3 &  ASTRAZENECA &    187547 &     26523 &    136.25 &   41.32 &   1.00 &   351.00 \\
3  &     idade\_paciente &  ASTRAZENECA &    214070 &         0 &     48.50 &   13.63 &      0 &      123 \\
4  &    mean\_income\_avg &  ASTRAZENECA &    181707 &     32363 &  3,118.86 & 1995.99 & 567.34 & 18553.01 \\
5  &  votos\_PSL-PT\_2018 &  ASTRAZENECA &    191328 &     22742 &      0.13 &    0.09 &  -0.35 &     0.50 \\
6  &            n\_doses &      JANSSEN &     22030 &         0 &      3.01 &    0.11 &      3 &        4 \\
7  &   diff\_dose1\_dose2 &      JANSSEN &     21805 &       225 &     71.34 &  

<ipython-input-31-eb7a20f18aa7>:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summary_table.to_latex())
